# Notebook 10 - Apprentissage par renforcement / Taxi autonome

CSI4106 Intelligence Artificielle   
Automne 2021  
Version 1 (2020) preparée par Julian Templeton, Caroline Barrière et Joel Muteba.  Version 2 (2021) adaptée par Caroline Barrière.

***INTRODUCTION***:  
Dans ce notebook, nous explorerons l'utilisation de l'apprentissage par renforcement pour aider un agent à résoudre une tâche spécifique dans un environnement fourni par la [bibliothèque OpenAI's Gym](https://gym.openai.com/). 

OpenAI's Gym propose de nombreuses expérimentations différentes à utiliser. Celles-ci vont d'exercices d'équilibre d'un bâton aux voitures autonomes en passant par le jeu simple d'Atari. Malheureusement, toutes les expérimentations disponibles ne peuvent pas être facilement utilisées. Beaucoup peuvent prendre des heures d'entraînement pour commencer à voir des résultats passionnants. Chacune de ces expérimentations utilise des agents qui peuvent être entraînés par apprentissage par renforcement pour maîtriser comment exécuter la tâche spécifiée. Les méthodes utilisées peuvent aller de la simple utilisation du Q-Learning à l'utilisation plus complexe d'un ou plusieurs modèles de Deep Learning qui fonctionnent en conjonction avec des techniques d'apprentissage par renforcement.

Dans ce notebook, nous explorerons un scénario dans lequel un taxi autonome situé dans une grille doit prendre un passager situé dans l'une de quatre positions et déposer le passager dans l'une de trois autres positions.

Pour vous familiariser avec le problème du **taxi autonome** abordé dans ce notebook, veuillez vous rendre sur le site https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/ et lire: \


*   section 1 (récompenses)
*   section 2 (espace d'états) qui vous fera comprendre pourquoi il y a 500 états possibles, 
*   section 3 (espace d'action) qui décrit les actions possibles.

Tout au long du notebook, nous travaillerons avec une approche de base aléatoire et une approche basée sur le Q-Learning. Cela fournira un aperçu de la façon dont Q-Learning peut être appliqué aux problèmes et comment un agent peut utiliser l'apprentissage par renforcement pour résoudre des problèmes dans un environnement.  La comparaison avec l'approche aléatoire montrera le potentiel du Q-Learning.

**Lors de la soumission de ce notebook, assurez-vous de NE PAS réinitialiser les sorties de l'exécution du code (et n'oubliez pas d'enregistrer le notebook avec ctrl + s).**

**Afin de faciliter l'installation, vous exécuterez à nouveau ce notebook dans Google Colab, PAS sur votre ordinateur local.**

***DEVOIR***:  
Parcourez le notebook en exécutant chaque cellule, une à la fois.
Recherchez **(TO DO)** pour les tâches que vous devez effectuer. Ne modifiez pas le code en dehors des questions auxquelles vous êtes invité à répondre à moins que cela ne vous soit spécifiquement demandé. Une fois que vous avez terminé, signez le notebook (à la fin du notebook), renommez-le *NumEtudiant-NomFamille-Notebook10.ipynb* et soumettez-le.

*Le notebook sera noté sur 30.  \
Chaque **(TO DO)** a un certain nombre de points qui lui sont associés.*
***

**1.0 - Mise en place du jeu de taxi**   

Pour commencer le notebook, nous devrons configurer et explorer l'environnement de Open Gym pour le taxi autonome.

Le taxi autonome doit prendre et déposer un passager dans un espace défini par une petite grille. Pour bien saisir l'environnement dans lequel le taxi autonome (l'agent) évolue, assurez-vous de lire les 3 sections du tutoriel sur les récompenses, les états et les actions, tel que mentionné dans l'introduction de ce notebook.

Le code utilisé dans tout le notebook provient de [cet exemple](https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/) et a été modifié conséquemment.

Pour commencer, nous installerons certains des packages dont nous aurons besoin pour exécuter le programme.

In [2]:
# Install the necessary libraries
!pip install cmake 'gym[atari]' scipy

In [3]:
# Import the necessary libraries
import random
import gym
import numpy as np
from IPython.display import clear_output

Avec toutes les bibliothèques installées, nous allons maintenant utiliser le programme Taxi fourni par Gym. Ci-dessous, nous allons importer Gym, charger le programme comme environnement actif et montrer une image représentant l'état actuel du programme.

D'après l'image ci-dessous, il existe quatre emplacements clés différents dans l'environnement, représentés par *R*, *G*, *Y* et *B*. La lettre qui est en gras en bleu représente l'endroit où le passager actuel doit être récupéré et la lettre en gras en violet représente l'endroit où le passager souhaite être déposé. Le bloc jaune représente la cellule dans laquelle se trouve actuellement le taxi. Par conséquent, le taxi doit d'abord récupérer le passager et le déposer au lieu désiré. Lorsqu'un passager est dans le taxi, la cellule jaune devient verte jusqu'à ce que le passager soit déposé.

In [4]:
# Load the environment
env = gym.make("Taxi-v3").env
# Render the current state of the program
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



Ensuite, nous allons réinitialiser l'état de l'environnement et montrer à nouveau l'état actuel. Nous imprimons également le nombre total d'actions disponibles pour notre agent (défini comme *l'espace d'action*) et *l'espace d'état* qui représente l'état du programme (où se trouve la cabine, le passager, le lieu de prise en charge et le lieu où déposer).

In [5]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


Nous voulons que notre agent apprenne quelle action entreprendre dans un état spécifique. Cet état dépend de l'emplacement du taxi par rapport à l'emplacement du passager et au lieu où déposer. Les six actions possibles que le taxi peut effectuer dans un état donné sont:

Action = 0: Aller vers le Sud

Action = 1: Aller vers le Nord

Action = 2: Aller vers l'Est

Action = 3: Aller vers l'Ouest

Action = 4: Prendre le passager

Action = 5: Déposer le passager

Vous trouverez ci-dessous un exemple de définition de l'état avec un codage spécifique et le rendu de cet état.

In [6]:
# The encoding below represents: (taxi row, taxi column, passenger index, destination index)
state = env.encode(3, 1, 2, 0) 
print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



L'exemple suivant montre comment définir un état avec le passager dans le taxi.

In [15]:
# The encoding below represents: (taxi row, taxi column, passenger index, destination index)
state = env.encode(0, 1, 4, 0) 
print("State:", state)

env.s = state
env.render()

State: 36
+---------+
|R:_| : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



**(TO DO) Q1 - 4 points**   
Maintenant que nous avons vu comment définir un état via un encodage, vous devrez définir l'état pour qu'il corresponde aux descriptions ci-dessous et les montrer.


**(TO DO) Q1 (a) - 2 points**    
Mettez le passager à la position G, le passager souhaitant être déposé à la position R.  Le taxi doit être positionné à un point aléatoire sur la grille (la position sélectionnée du taxi doit être choisie au hasard). Après avoir défini la position, montrez le rendu (avec render) de l'état.

In [70]:
# RÉPONSE Q1(a) 
# remember to use random coordinates within the grid for the taxi...
state1 = env.encode(random.randint(0, 4), random.randint(0, 4), 1, 0) 
print("State:", state)

env.s = state1
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


**(TO DO) Q1 (b) - 2 points**

Mettez le passager pour qu'il soit dans le taxi (à n'importe quelle position sans lettre dessus) et mettez le point de débarquement des passagers sur la position B. Après avoir défini la position, montrer le rendu (avec render) l'état.

In [71]:
# RÉPONSE Q1(b)
state2 = env.encode(random.randint(0, 4), random.randint(0, 4), 4, 3) 
print("State:", state)

env.s = state2
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : :_: |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


Pour chaque action que le taxi peut entreprendre, nous avons une liste représentant les informations clés par rapport à ce qui se passera lorsqu'une action est effectuée. Après avoir effectué une action, l'agent recevra une récompense ou une pénalité et l'environnement se retrouvera dans un nouvel état.

Ci-dessous, nous affichons un dictionnaire (similaire à la table de récompense - reward table - présentée dans la vidéo du cours) qui contient toutes les actions possibles ainsi que les informations suivantes dans les tuples correspondants:

(

   La probabilité de prendre cette action,
  
   L'état résultant après avoir effectué cette action,
  
   La récompense pour avoir pris cette action,
  
   Si le programme se terminera ou non lors de l'exécution de l'action

)

Exemple de tuple: (1.0, 328, -1, False)

In [44]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

Bien que non affiché par le code ci-dessus, si le taxi contient le passager et est au-dessus du point de débarquement, la récompense pour l'action de déposer le passager est de 20.

**2.0 - Approche de base du jeu du taxi**   

Pour commencer, nous allons effectuer la simulation du scénario de taxi avec une approche de base qui n'utilise pas Q-Learning. Cette approche fonctionnera simplement en sélectionnant une action aléatoire disponible à chaque pas de temps, quel que soit l'état actuel. Nous préparerons également une méthode de génération de toutes les images d'un épisode pour voir comment l'agent contrôle le taxi dans le scénario.

In [45]:
def run_single_simulation_baseline(env, state, disable_prints=False):
    '''
    Given the environment and a specific state, randomly select an action for the taxi
    to perform until the goal is completed.
    '''
    if not disable_prints:
        print("Testing for simulation: {}".format(state))
    # Set the state of the environment
    env.s = state
    # Used to hold all information for a single time step (including the image data)
    frames = []
    # Used to determine when the simulation has been completed
    done = False
    # Determines the number of times steps that the application has been run for
    time_steps = 0
    # The total values used to determine how many times the agent mistakenly
    # picks up no one or attempts to dropoff no passenger or attempts to
    # dropoff a passenger in the wrong position.
    penalties, reward = 0, 0
    # Run until the passenger has been picked up and dropped off in the target location
    while not done:
        # Perform a random action from the set of available actions in the environment
        action = env.action_space.sample()
        # From performing the action, retrieve the new state, the reward from taking the action,
        # whether the simulation is complete, and other information from performing the action.
        state, reward, done, info = env.step(action)
        # If an incorrect dropoff or pickup is performed, increment the penalty count
        if reward == -10:
            penalties += 1
        # Put each rendered frame into dict to use for animating the process and
        # tracking the details over the run
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward
            }
        )
        # Increment the time step count
        time_steps += 1
    # State the total number of steps taken and the total penalties that have occured.
    if not disable_prints:
        print("Timesteps taken: {}".format(time_steps))
        print("Penalties incurred: {}".format(penalties))
    # Return the frame data, the total penalties, and the total time steps
    return frames, penalties, time_steps

Une fois l'approche de base définie, nous allons exécuter un test avec cette approche pour voir combien de temps il faut à un agent utilisant cette approche pour trouver une solution pour la simulation 328 et combien de pénalités majeures l'agent reçoit.

In [64]:
state = 328
# Run a test and collect all frames from the run
frames, _, _ = run_single_simulation_baseline(env, state)

Testing for simulation: 328
Timesteps taken: 435
Penalties incurred: 137


Après avoir effectué une simulation et récupéré les résultats, nous pouvons utiliser les images obtenues à partir de la simulation et les transmettre à la fonction *print_frames* ci-dessous pour afficher une animation contenant toutes les images ainsi que les informations qui provenaient de chaque étape et temps auquel l'image correspond .

Pour le premier épisode que vous visualisez, il est recommandé d'exécuter l'ensemble du processus à une vitesse plus lente (telle que 0.3 ou 0.5 pour le paramètre sleepTime). Cependant, vous êtes libre d'augmenter la vitesse du processus par la suite.

In [66]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames, sleepTime=0.3):
    '''
    For each frame, show the frame and display the timestep it occurred at,
    the number of the active state, the action selected, adn the corresponding reward.
    '''
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        # Can adjust speed here
        sleep(sleepTime)

In [69]:
# Print the frames from the episode
print_frames(frames, 0.1)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 435
State: 0
Action: 5
Reward: 20


**(TO DO) Q2 - 2 points**

Regardons quelques simulations d'une stratégie aléatoire.  Pour ce faire, vous débuterez avec les états que vous avez établis pour Q1.

**(TO DO) Q2 (a) - 1 point**   
En utilisant l'état défini à partir de Q1 (a), récupérer les frames/images correspondantes obtenues en utilisant l'approche de base ci-dessus. Ensuite, affichez ces cadres(frames ou images).

In [72]:
# RÉPONSE Q2(a)
# Retrieve the corresponding frames from running the simulation starting from the state found in Q1 (a). Then show those frames.
frames1, _, _ = run_single_simulation_baseline(env, state1)
print_frames(frames1, 0.1)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 1903
State: 0
Action: 5
Reward: 20


**(TO DO) Q2(b) - 1 point**

a) En utilisant l'état défini à partir de Q1 (b), récupérer les frames/images correspondantes obtenues en utilisant l'approche de base ci-dessus. Ensuite, affichez ces cadres(frames ou images).

In [73]:
# RÉPONSE Q2(b)
# Retrieve the corresponding frames from running the simulation starting from the state found in Q1 (b). Then show those frames.
frames2, _, _ = run_single_simulation_baseline(env, state2)
print_frames(frames2, 0.1)


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 26
State: 475
Action: 5
Reward: 20


Avec la possibilité de simuler des exécutions uniques d'un épisode avec l'approche de base, nous allons maintenant définir une fonction que nous utiliserons pour évaluer les performances générales du modèle de base lorsqu'il est exécuté sur plusieurs épisodes. La fonction *evaluate_agent_baseline* ci-dessous accepte comme entrée le nombre total d'épisodes sélectionnés au hasard à exécuter avec l'environnement, exécute les épisodes aléatoires, affiche le nombre moyen de pas de temps pris par épisode avec les pénalités moyennes encourues et renvoie les données d'image.



In [74]:
def evaluate_agent_baseline(episodes, env):
    '''
    Given a number of episodes and an environment, run the specified
    number of episodes, where each run begins with a random state, display the
    naverage timesteps per episode and the average penalties per episode, and output
    the frames to be displayed.
    '''
    total_time_steps, total_penalties = 0, 0
    frames = []
    # Run through the total number of episodes
    for _ in range(episodes):
        # Get a random state
        state = env.reset()
        # Run the simulation, obtaining the results
        frame_data, penalties, time_steps = run_single_simulation_baseline(env, state, True)
        # Update the tracked data over all simulations
        total_penalties += penalties
        total_time_steps += time_steps
        frames = frames + frame_data
    print(f"Results after {episodes} episodes:")
    print(f"Average timesteps per episode: {total_time_steps / episodes}")
    print(f"Average penalties per episode: {total_penalties / episodes}")
    return frames

**(TO DO) Q3 - 5 points**

Évaluons la méthode de base aléatoire.  Pour ce faire, nous devons exécuter un certain nombre d'épisodes pour faire une moyenne des résultats.

**(TO DO) Q3(a) - 1 point**

Utilisez la fonction *evaluate_agent_baseline* définie ci-dessus pour exécuter 100 épisodes aléatoires pour l'environnement.

***Si la fonction evaluate_agent_baseline semble s'exécuter trop longtemps (plusieurs minutes, pas une seule), arrêtez l'exécution en cliquant sur le bouton en haut à gauche de la cellule de code en cours d'exécution et réexécutez-la.***

In [76]:
# RÉPONSE Q3(a) 
evaluate_agent_baseline(100, env)


Results after 100 episodes:
Average timesteps per episode: 2574.47
Average penalties per episode: 833.19


[{'action': 0,
  'frame': '+---------+\n|R: | : :\x1bG\x1b|\n| : | : : |\n| : : : : |\n| | : | : |\n|\x1bY\x1b| : |B:\x1b \x1b|\n+---------+\n  (South)\n',
  'reward': -1,
  'state': 486},
 {'action': 5,
  'frame': '+---------+\n|R: | : :\x1bG\x1b|\n| : | : : |\n| : : : : |\n| | : | : |\n|\x1bY\x1b| : |B:\x1b \x1b|\n+---------+\n  (Dropoff)\n',
  'reward': -10,
  'state': 486},
 {'action': 5,
  'frame': '+---------+\n|R: | : :\x1bG\x1b|\n| : | : : |\n| : : : : |\n| | : | : |\n|\x1bY\x1b| : |B:\x1b \x1b|\n+---------+\n  (Dropoff)\n',
  'reward': -10,
  'state': 486},
 {'action': 1,
  'frame': '+---------+\n|R: | : :\x1bG\x1b|\n| : | : : |\n| : : : : |\n| | : | :\x1b \x1b|\n|\x1bY\x1b| : |B: |\n+---------+\n  (North)\n',
  'reward': -1,
  'state': 386},
 {'action': 0,
  'frame': '+---------+\n|R: | : :\x1bG\x1b|\n| : | : : |\n| : : : : |\n| | : | : |\n|\x1bY\x1b| : |B:\x1b \x1b|\n+---------+\n  (South)\n',
  'reward': -1,
  'state': 486},
 {'action': 3,
  'frame': '+---------+\n|R: | : :

**(TO DO) Q3(b) - 2 points**

D'après les résultats de la Q3 (a), comment l'approche de référence a-t-elle fonctionné et pourquoi pensez-vous qu'elle a bien ou mal fonctionné? Expliquez en référent aux temps moyens par épisode et aux pénalités moyennes par épisode.

**RÉPONSE Q3(b)**   

Cette approche à mal fonctionner, car le temps moyen par épisode est de 2574.47 et les pénalités moyennes sont de 833.19 par épisode. Cette approche n'a pas bien fonctionner, car elle est basé sur la chance, des états aléatoires et des actions aléatoires. De plus, cette méthode n'apprend pas.


**(TO DO) Q3(c) - 2 points**

Sans suggérer d'utiliser une approche d'apprentissage par renforcement (comme nous le ferons dans la partie 3), suggérez 2 idées qui pourraient être incluses dans une approche de base améliorée pour lui permettre de fonctionner légèrement mieux ?

**RÉPONSE Q3(c)**

1: On pourrait continuer de choisir un état aléatoire sauf qu'au lieu de choisir une action aléatoire on pourrait parcourir la colonne de l'état de haut en bas.

2: On pourrait continuer de choisir un état aléatoire sauf qu'au lieu de choisir une action aléatoire on pourrait parcourir la rangée de l'état de gauche à droite.


**3.0 - Entraîner un agent avec Q-Learning pour jouer au jeu du taxi**   

Maintenant que nous avons vu une simulation du taxi autonome utilisant une stratégie aléatoire, nous nous tournons vers l'apprentissage par renforcement pour obtenir une meilleure stratégie par l'intermédiaire de l'algorithme Q-Learning.

Pour démarrer le processus, nous allons créer une table de valeurs Q pour chaque possibilité d'état d'action (en l'initialisant à zéro). L'agent mettra à jour cette table lors de l'entraînement et *ATTENTION*, il aura besoin de la ***réinitialiser*** chaque fois qu'il souhaite réinitialiser son entraînement.

In [77]:
# Initialize the table of Q values for the state-action pairs
q_table = np.zeros([env.observation_space.n, env.action_space.n])

Une fois la Q-Table initialisée, nous allons maintenant définir la fonction d'apprentissage qui ajuste les valeurs Q dans *q_table*. Le processus d'apprentissage consiste à exécuter un certain nombre de simulations aléatoires et à mettre à jour les valeurs Q pour chaque état via Q-Learning.

Il existe un certain nombre d'hyperparamètres utilisés par la fonction d'apprentissage:

- *alpha*: paramètre d'apprentissage 
- *gamma*: le paramètre de *discount* sur le gain à long terme.
- *epsilon*: paramètre d'Exploitation/Exploration
- *num_simulations*: nombre d'épisodes à générer pour que le taxi autonome mette à jour ses valeurs Q.

Ainsi, en exécutant cet algorithme, un agent peut apprendre les valeurs Q à utiliser lorsqu'il sera par la suite mis en situation de test.

In [78]:
def train_agent(alpha, gamma, epsilon, num_simulations):
    '''
    Trains an agent by updating its Q values for a total of num_simulations
    episodes with the alpha, gamma, and epsilon hyperparameters. 
    '''
    # For plotting metrics
    all_time_steps = []
    all_penalties = []
    # Generate the specified number of episodes
    for i in range(1, num_simulations + 1):
        # Generate a new state by resetting it
        state = env.reset()
        # Variables tracked (time steps, total penalties, the reward value)
        time_steps, penalties, reward, = 0, 0, 0
        done = False
        # Run the simulation 
        while not done:
            # Select a random action is the randomly selected number from a
            # uniform distribution is less than epsilon
            if random.uniform(0, 1) < epsilon:
                action = env.action_space.sample() # Explore action space
            # Otherwise use the currently learned Q values
            else:
                action = np.argmax(q_table[state]) # Exploit learned values
            # Retrieve the relevant information after performing the action
            next_state, reward, done, info = env.step(action) 
            # Retrieve the old Q value and the maximum Q value from the next state
            old_value = q_table[state, action]
            next_max = np.max(q_table[next_state])
            # Update the current Q value
            new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
            q_table[state, action] = new_value
            # Track anytime an incorrect dropoff or pickup is made
            if reward == -10:
                penalties += 1
            # Proceed to the next state and time step
            state = next_state
            time_steps += 1
        # Display progress for each 100 episodes
        if i % 100 == 0:
            clear_output(wait=True)
            print(f"Episode: {i}")
    print("Training finished.\n")

Nous utilisons maintenant la fonction d'entraînement avec un ensemble d'hyperparamètres pour entraîner l'agent avec Q-Learning afin d'améliorer potentiellement les performances au fil du temps.  Le nombre d'épisodes est établi à 100000, donc l'apprentissage va prendre un peu de temps.

In [79]:
# Hyperparameters
alpha = 0.1
gamma = 0.5
epsilon = 0.1
num_simulations = 100000
# Train the agent
train_agent(alpha, gamma, epsilon, num_simulations)

Episode: 100000
Training finished.



Après l'entraînement, nous pouvons regarder les valeurs Q qui ont été obtenues dans notre table state-action pour un état spécifique. Ci-dessous, nous voyons que chaque valeur Q pour les six actions possibles disponibles pour l'état 328 a été mise à jour en conséquence.

**(TO DO) Q4 - 2 points**

Ci-dessous, nous imprimons les valeurs Q disponibles pour les six actions à l'état 328 et nous montrons cet état pour le visualiser. Sur la base des valeurs Q disponibles (en supposant que nous sommes en mode d'exploitation), quelle action serait la prochaine à être sélectionnée (ou s'il y a des égalités, listez toutes les actions possibles qui seraient envisagées)? Certaines des actions contenant des valeurs Q plus élevées semblent-elles problématiques si jamais elles sont sélectionnées? Pourquoi ou pourquoi pas?

In [80]:
print(q_table[328])
env.s = 328
env.render()

[ -1.98416308  -1.95703125  -1.98607779  -1.97636037 -10.54987212
  -9.97802208]
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


**RÉPONSE Q4**

La prochaine action serait d'aller vers le Nord puisqu'elle a la valeur la plus élevé. Certaines valeurs sont très proches de la valeur maximale (nord). Avec une mauvaise évaluation, cela pourrait causer une mauvaise décision si elles sont sélectionné.


Une fois l'entraînement terminé, nous pouvons maintenant évaluer l'approche Q-Learning d'une façon similaire à l'évaluation de l'approche de base. En passant le nombre d'épisodes à tester et l'environnement, nous générons ce nombre d'épisodes aléatoires et faisons la moyenne des résultats obtenus en exécutant l'approche Q-Learning pour effectuer les épisodes. Contrairement à l'entraînement, il est important de noter que les hyperparamètres qui étaient utilisés ne sont plus utilisés ici. L'agent utilise simplement la valeur Q maximale à chaque étape pour déterminer quelle action entreprendre à un pas de temps donné.  Il s'agit en fait d'une stratégie gloutonne (et non epsilon-gloutonne) qui suit uniquement la valeur de Q maximale.  Nous testons donc en mode d'exploitation (et non exploration/exploitation).



In [81]:
def evaluate_agent_QL(episodes, env):
    '''
    Given a number to specify how many random states to run and the environment to use,
    display the averaged metrics obtained from the tests and return the frames obtained from the tests.
    '''
    total_time_steps, total_penalties = 0, 0
    frames = []
    for _ in range(episodes):
        # Generate a random state to use
        state = env.reset()
        # The information collected throughout the run
        time_steps, penalties, reward = 0, 0, 0
        # Determines when the episode is complete
        done = False
        # Run through the episode until complete
        while not done:
            # Select the action containing the maximum Q value
            action = np.argmax(q_table[state])
            # Run that action and retrieve the reward and other details
            state, reward, done, info = env.step(action)

            # Put each rendered frame into dict for animation
            frames.append({
                'frame': env.render(mode='ansi'),
                'state': state,
                'action': action,
                'reward': reward
                }
            )
            # Specify whether the agent incorrectly chose to pick up or dropoff a passenger
            if reward == -10:
                penalties += 1
            # Increment the current time step
            time_steps += 1
        # Track the totals
        total_penalties += penalties
        total_time_steps += time_steps
    # Display the performance over the tests
    print(f"Results after {episodes} episodes:")
    print(f"Average timesteps per episode: {total_time_steps / episodes}")
    print(f"Average penalties per episode: {total_penalties / episodes}")
    # Return the frames to allow a user to view the runs
    return frames

**(TO DO) Q5 - 3 points**

Il y a un aspect aléatoire dans Q-learning, il faut donc courir un certain nombre d'épisodes pour donner un résultat moyen. Fixons ce nombre suffisamment haut pour que la moyenne soit significative. Voyons comment Q-learning se comporte par rapport à l'approche de base.

***ATTENTION: Si la fonction evaluate_agent_QL semble s'exécuter trop longtemps (30s ou plus), arrêtez l'exécution en cliquant sur le bouton en haut à gauche de la cellule de code en cours d'exécution et réexécutez-la. Cela se produit parce que l'entraînement était suffisant pour définir des valeurs Q valides et a entraîné une impasse pour un état spécifique.***

**(TO DO) Q5 (a) - 1 point**

Exécutez *evaluate_agent_QL* pendant 500 épisodes pour récupérer le nombre moyen de pas de temps et la pénalité moyenne après l'entraînement.

In [83]:
# RÉPONSE Q5(a)
evaluate_agent_QL(500, env)

Results after 500 episodes:
Average timesteps per episode: 13.156
Average penalties per episode: 0.0


[{'action': 3,
  'frame': '+---------+\n|\x1b\x1bR\x1b\x1b: | : :G|\n| : | : : |\n| : : : : |\n| | : | : |\n|\x1bY\x1b| : |B: |\n+---------+\n  (West)\n',
  'reward': -1,
  'state': 2},
 {'action': 4,
  'frame': '+---------+\n|\x1bR\x1b: | : :G|\n| : | : : |\n| : : : : |\n| | : | : |\n|\x1bY\x1b| : |B: |\n+---------+\n  (Pickup)\n',
  'reward': -1,
  'state': 18},
 {'action': 0,
  'frame': '+---------+\n|R: | : :G|\n|\x1b_\x1b: | : : |\n| : : : : |\n| | : | : |\n|\x1bY\x1b| : |B: |\n+---------+\n  (South)\n',
  'reward': -1,
  'state': 118},
 {'action': 0,
  'frame': '+---------+\n|R: | : :G|\n| : | : : |\n|\x1b_\x1b: : : : |\n| | : | : |\n|\x1bY\x1b| : |B: |\n+---------+\n  (South)\n',
  'reward': -1,
  'state': 218},
 {'action': 0,
  'frame': '+---------+\n|R: | : :G|\n| : | : : |\n| : : : : |\n|\x1b_\x1b| : | : |\n|\x1bY\x1b| : |B: |\n+---------+\n  (South)\n',
  'reward': -1,
  'state': 318},
 {'action': 0,
  'frame': '+---------+\n|R: | : :G|\n| : | : : |\n| : : : : |\n| | : | : |

**(TO DO) Q5 (b) - 2 points**

Compte tenu de vos résultats de Q5 (a), comment les résultats observés des tests se comparent-ils aux tests de l'approche de base de Q3 (a)? Plus précisément, quelle stratégie est la plus performante par rapport au nombre moyen de pénalités tout au long des tests et quelle stratégie est capable de mener les passagers du point de départ à la destination plus rapidement (en moyenne).

**RÉPONSE Q5(b)**

La stratégie Q-Learning est la plus efficace avec 0.0 de pénalité moyenne et un nombre de pas moyen pas épisode de 13.156. Cette stratégie est beaucoup plus performante que celle de base avec un nombre de pas moyen par épisode de 2574.47 et une pénalité moyenne de 833.19. 


**4.0 - Test de différents hyperparamètres**   

Nous allons maintenant essayer de re-entraîner l'agent en utilisant différentes configurations avec les hyperparamètres. Cela vous permettra d'explorer leur impact sur le Q-Learning et de comprendre leur objectif pendant l'entraînement.  

**(TO DO) Q6 - 12 points**     
Ci-dessous, nous explorons différentes variations de quatre hyperparamètres utilisés par l'approche Q-Learning pour mieux comprendre leur impact sur l'entraînement. Lorsque vous répondez aux questions, ***veillez à régler correctement les hyperparamètres***.

A noter, ci-dessous sont les valeurs d'hyperparamètres initiales utilisées à la section 3.0 de ce notebook à utiliser comme référence:

*alpha* = 0,1

*gamma* = 0,5

*epsilon* = 0,1

*num_simulations* = 100000

Pour toutes nos évaluations, nous utiliserons 500 épisodes. De plus, après chaque expérience, vous imprimerez la valeur qtable pour l'état 328, afin de pouvoir la comparer avec ce qui a été obtenu dans la question Q4 plus tôt.

***ATTENTION (répété ici de Q5): Si la fonction evaluate_agent_QL semble s'exécuter trop longtemps (30s ou plus), arrêtez l'exécution en cliquant sur le bouton en haut à gauche de la cellule de code en cours d'exécution et réexécutez-la. Cela se produit parce que l'entraînement était suffisant pour définir des valeurs Q valides et a entraîné une impasse pour un état spécifique.***

**(TO DO) Q6 (a) - 1.5 points**

Re-entraînez l'agent en réinitialisant les valeurs d'apprentissage Q et l'entraînement pour seulement **35000 épisodes** (avec les mêmes valeurs alpha, gamma et epsilon utilisées dans la section 3.0 de ce notebook). Ensuite, effectuez un test pour 500 épisodes dans l'environnement.  Imprimez la valeur de *q_table* pour l'état 328.

In [84]:
# RÉPONSE Q6(a)
# TODO: Reset q_table
q_table = np.zeros([env.observation_space.n, env.action_space.n])
# TODO: Retrain with the specified hyperparameters
# Hyperparameters
alpha = 0.1
gamma = 0.5
epsilon = 0.1
num_simulations = 35000
train_agent(alpha, gamma, epsilon, num_simulations)
# TODO: Test for 500 episodes
evaluate_agent_QL(500, env)
# TODO: Print q_table for state 328
print(q_table[328])
env.s = 328
env.render()

Episode: 35000
Training finished.

Results after 500 episodes:
Average timesteps per episode: 13.15
Average penalties per episode: 0.0
[-1.96968092 -1.95703125 -1.97695721 -1.96299821 -7.06980823 -9.42977307]
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


**(TO DO) Q6 (b) - 1.5 points**

Re-entraînez l'agent en réinitialisant les valeurs d'apprentissage Q et en s'entraînant pour **100000 épisodes**, mais avec une **valeur epsilon de 0,8** (avec les mêmes valeurs alpha et gamma utilisées dans la section 3.0 de ce notebook). Ensuite, effectuez un test pour 500 épisodes dans l'environnement.  Imprimez la valeur de *q_table* pour l'état 328.

In [85]:
# RÉPONSE Q6(b)
# TODO: Reset q_table
q_table = np.zeros([env.observation_space.n, env.action_space.n])
# TODO: Retrain with the specified hyperparameters
# Hyperparameters
alpha = 0.1
gamma = 0.5
epsilon = 0.8
num_simulations = 100000
train_agent(alpha, gamma, epsilon, num_simulations)
# TODO: Test for 500 episodes
evaluate_agent_QL(500, env)
# TODO: Print q_table for state 328
print(q_table[328])
env.s = 328
env.render()

Episode: 100000
Training finished.

Results after 500 episodes:
Average timesteps per episode: 13.02
Average penalties per episode: 0.0
[ -1.98925781  -1.95703125  -1.98925781  -1.97851563 -10.97851563
 -10.97851563]
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


**(TO DO) Q6 (c) - 1.5 points**

Re-entraînez l'agent en réinitialisant les valeurs d'apprentissage Q et en s'entraînant pour **100000 épisodes**, mais avec une **valeur alpha de 0,7** (avec les mêmes valeurs gamma et epsilon utilisées dans la section 3.0 de ce notebook). Ensuite, effectuez un test pour 500 épisodes dans l'environnement.  Imprimez la valeur de *q_table* pour l'état 328.

In [86]:
# RÉPONSE Q6(c)
# TODO: Reset q_table
q_table = np.zeros([env.observation_space.n, env.action_space.n])
# TODO: Retrain with the specified hyperparameters
# Hyperparameters
alpha = 0.7
gamma = 0.5
epsilon = 0.1
num_simulations = 100000
train_agent(alpha, gamma, epsilon, num_simulations)
# TODO: Test for 500 episodes
evaluate_agent_QL(500, env)
# TODO: Print q_table for state 328
print(q_table[328])
env.s = 328
env.render()

Episode: 100000
Training finished.

Results after 500 episodes:
Average timesteps per episode: 13.098
Average penalties per episode: 0.0
[ -1.98925775  -1.95703125  -1.98925781  -1.97851563 -10.97851543
 -10.97851562]
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


**(TO DO) Q6 (d) - 1.5 points**

Re-entraînez l'agent en réinitialisant les valeurs d'apprentissage Q et en entraînant pour **100000 épisodes**, mais avec une **valeur gamma de 0,15** (avec les mêmes valeurs alpha et epsilon utilisées dans la section 3.0 de ce notebook). Ensuite, effectuez un test pour 500 épisodes dans l'environnement.  Imprimez la valeur de *q_table* pour l'état 328.

In [87]:
# RÉPONSE Q6(d)
# TODO: Reset q_table
q_table = np.zeros([env.observation_space.n, env.action_space.n])
# TODO: Retrain with the specified hyperparameters
# Hyperparameters
alpha = 0.1
gamma = 0.15
epsilon = 0.1
num_simulations = 100000
train_agent(alpha, gamma, epsilon, num_simulations)
# TODO: Test for 500 episodes
evaluate_agent_QL(500, env)
# TODO: Print q_table for state 328
print(q_table[328])
env.s = 328
env.render()

Episode: 100000
Training finished.

Results after 500 episodes:
Average timesteps per episode: 13.072
Average penalties per episode: 0.0
[-1.17647051 -1.17646977 -1.1764705  -1.1764704  -9.92080833 -9.78682968]
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


**(TO DO) Q6 (e) - 6 points**

En utilisant les résultats obtenus à partir de vos tests en Q6 (a), (b), (c) et (d), ainsi que les résultats initiaux de Q5 (approche *baseline*), expliquez les impacts de la modification du nombre d'épisodes entraînés (moins d'épisodes vs plus d'épisodes), la valeur alpha (petite vs grande), la valeur gamma (petite vs grande) et la valeur epsilon (petite vs grande). Même si les différences dans les comparaisons sont mineures, indiquez-les.

En plus de regarder les résultats obtenus (nombre moyen de *timestep*), discutez également de l'impact sur les q-values des différentes actions de l'état 328. Est-ce que celles-ci ont changé ?


**RÉPONSE Q6(e)**

*Assurez-vous de discuter de chacun des hyperparamètres en lien avec le nombre de timestep et les q-values.*

En diminuant le nombre d'épisodes d'entrainement soit de 100000 à 35000 on obtient un nombre de timestep très similaire parfois même plus performant. Par contre, avec moins d'entrainement, on remarque que les q-values sont moins précise, c'est-à-dire qu'elles se rapproche plus de la valeur maximale, soit le nord dans ce cas-ci. Avec 35000 épisodes d'entrainement on a quand même des q-values assez juste. Même avec plus d'épisodes les q-values ne changent pas beaucoup. Ceci est dû au fait que l'apprentissage se fait à plus petit pas vers la fin qu'au début.

En ayant une valeur alpha plus grande, on remarque que le nombre de timestep devient un peu plus petit. De plus, les q-values deviennent plus précises, car on remarque des plus grands écarts entre la meilleur décision et les autres décisions. Ceci est dû au fait que alpha est le learning rate. Avec  un plus grand learning rate, le model apprend plus rapidement, donc effectue plus de changements.

Avec une valeur gamma plus petite, on remarque une petite diminution dans le nombre de timestep. Par contre, les q-values deviennent beaucoup moins précises. On remarques que toutes les valeurs sont sensiblement les mêmes. Un petit écart entre les q-values, peut causer une mauvaise action. Ceci est dû au fait que gamma est la valeur de la récompense future. Avec un plus petit gamma, l'agent considérera les récompense immédiates par dessus les récompenses futures. 

Une valeur epsilon plus grande cause une réduction du nombre de timestep. De plus, les q-values deviennent plus précises, car on remarque des plus grands écarts entre la meilleur décision et les autres décisions. Epsilon représente les actions aléatoires. Avec un plus grand epsilon, on introduit des actions aléatoire pour explorer. Dans ce cas-ci, mes résultats peuvent être dû à la chance puisque que j'ai obtenue un nombre de timestep plus petit et des q-values plus précises.


**5.0 - Vers un environnement de plus grande taille**   

Comme nous l'avons vu dans les vidéos du cours, et comme vous l'avez constaté ici, une limitation de Q-Learning est que les Q-Values doivent être apprises pour toutes les paires état/action. Dans notre problème-jouet du taxi autonome, nous avons travaillé avec une petite grille (5x5) et seulement 4 lieux pour prendre et déposer un passager, et nous étions déjà à 500 états, donc avec 6 actions, cela faisait 3000 paires état/action, donc 3000 q-values à apprendre.

Imaginez une grille plus grande de 100x100, et imaginez que toutes les cellules de la grille pourraient être des cellules où prendre ou déposer un passager, alors Q-learning ne serait pas une approche réaliste et nous devrions passer à Deep Q-Learning, comme nous l'avons vu dans le vidéos de cours.

La mise en œuvre d'une approche Deep Q-Learning dépasse le cadre de ce Notebook, mais nous pouvons tout de même penser au type d'architecture que cela nécessiterait.


**(TO DO) Q7 - 2 points**

Compte tenu de cette nouvelle description d'une grille plus grande et d'un ensemble plus vaste de lieux pour prendre et déposer un passager, décrivez à quoi pourrait ressembler un réseau de neurones pour le Deep-Q-learning. Que pourrait-on mettre aux nœuds d'entrée et combien en faudrait-il ? Que devrions-nous apprendre aux nœuds de sortie et combien de nœuds de sortie seraient nécessaires?

**RÉPONSE Q7**

Les noeuds d'entrée sont le nombre de positions. Avec une grille de 100x100 on peut donc avoir 10 000 positions/entrées. Dans chaque position, on a les possibilités soit un lieux pour prendre un passager, un lieux pour déposer un passager ou un lieux qui est seulement un lieux ordinaire, soit une route. 

Les noeuds de sortit représente les 6 différentes actions, donc on a 6 noeuds de sortie. On veux apprendre la bonne action à ces noeuds de sortie, soit sud, nord, est, ouest, prendre le passager et déposer le passager.


***SIGNATURE:***
Mon nom est Mark-Olivier Poulin.
Mon numéro d'étudiant(e) est 300058025.
Je confirme être l'auteur(e) de ce travail.